In [2]:
import numpy as np
import re
import random
import pandas as pd

Turn string into a list

In [67]:
def textParse(input_string):
    listofTokens = re.split(r'\W+',input_string)
    return [tok.lower() for tok in listofTokens if len(listofTokens) > 2]

Create a Vocabulary List

In [28]:

def createVocablist(doclist):
    vocabSet = set([])
    for document in doclist:
        vocabSet = vocabSet|set(document)
    return list(vocabSet)

Create a Vector

In [29]:
def setOfWord2Vec(vocabList, doc):
    returnVec = [0]*len(vocabList)
    for word in doc:
        if word in vocabList:
            returnVec[vocabList.index(word)] = 1
    return returnVec

Train Naive Bayes Model

In [50]:
def trainNB(trainMat, trainClass):
    numDocs = len(trainMat)
    numWords = len(trainMat[0])
    p_Spam = trainClass.count('spam')/float(len(trainClass))
    p_Ham = 1-p_Spam
    Ham_num = np.ones((numWords))
    Spam_num = np.ones((numWords))
    Ham_denom = 2
    Spam_denom = 2

    for i in range(numDocs):
        if trainClass[i] == 'spam':
            Spam_num += trainMat[i]
            Spam_denom += sum(trainMat[i])
        else:
            Ham_num += trainMat[i]
            Ham_denom += sum(trainMat[i])
    Spam_vec = np.log(Spam_num/Spam_denom)
    Ham_vec = np.log(Ham_num/Ham_denom)
    return Spam_vec, Ham_vec, p_Spam, p_Ham

Prediction

In [44]:
def classifyNB(wordVec,Spam_vec, Ham_vec, p_Spam, p_Ham):
    p1 = np.log(p_Spam) + sum(wordVec*Spam_vec)
    p0 = np.log(p_Ham) + sum(wordVec*Ham_vec)
    if p0 > p1:
        return 'ham'
    else:
        return 'spam'


Main Code

In [68]:
def spam():
    df = pd.read_csv("spam.csv")
    msk = np.random.rand(len(df)) < 0.8
    train = df[msk]
    test = df[~msk]
    vocabList = createVocablist([textParse(str) for str in list(df['Message'])])
    trainDoc = [textParse(str) for str in list(train['Message'])]
    testDoc = [textParse(str) for str in list(test['Message'])]
    trainClass = list(train['Category'])
    testClass = list(test['Category'])
    trainMat = []
    for doc in trainDoc:
        trainMat.append(setOfWord2Vec(vocabList,doc))
    Spam_vec, Ham_vec, p_Spam, p_Ham = trainNB(np.array(trainMat),trainClass)
    errorCount = 0
    for i in range(len(testDoc)):
        wordVec = setOfWord2Vec(vocabList,testDoc[i])
        prediction = classifyNB(np.array(wordVec), Spam_vec, Ham_vec, p_Spam, p_Ham)
        if testClass[i] != prediction:
            errorCount += 1
    accuracy = 1 - errorCount/len(testClass)
    print(accuracy)


In [69]:
spam()

0.9686907020872866
